<a href="https://colab.research.google.com/github/Kostratana/logistic_optimization_genetic_ants_algorithms/blob/main/Distance_matrix_ant_algorithm_%22Aidigo%22_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install osmnx

import osmnx as ox # Маршруты на графах
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # Отрисовка изображений
import seaborn as sns # Надстройка визуализации над matplotlib
import networkx as nx # Для анализа и визуализации графовых структур
import folium # Для создания интерактивных карт
from itertools import cycle # Для создания итератора
from numpy import radians
import numpy as np
from scipy import spatial
import time
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [ ]:
!pip -q install sweetviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 50.6 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Рейсы20042023.xlsx to Рейсы20042023.xlsx


In [ ]:
!pip install scipy

In [ ]:
!pip install haversine

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Авторизация в Google Диске
gauth = GoogleAuth()
drive = GoogleDrive(gauth)

In [ ]:
# Загрузка файла Excel
df = pd.read_excel('/content/Рейсы20042023.xlsx')

# Удаление столбцов
df.drop(['КОЛИЧЕСТВО_ПАЛЕТ', 'УИД_ВОДИТЕЛЯ','УИД_ДОКУМЕНТА','ВЕС_ДОКУМЕНТА','ОБЪЁМ_ДОКУМЕНТА','КОЛ_ВО_ПАЛЕТ_ДОКУМЕНТА','ПОРЯДОК_ЗАГРУЗКИ','УИД_РЕЙСА','УИД_АВТОМОБИЛЯ','ВЕС','ОБЪЕМ'], axis=1, inplace=True)
# Сохранение изменений в новом файле Excel
df.to_excel('coordinates.xlsx', index=False)
df.head


<bound method NDFrame.head of        ДОЛГОТА     ШИРОТА
0    60.881252  56.782632
1    60.881252  56.782632
2    60.881252  56.782632
3    60.881252  56.782632
4    60.881252  56.782632
..         ...        ...
568  60.601138  56.870514
569  60.601138  56.870514
570  60.601138  56.870514
571  60.601138  56.870514
572  60.601138  56.870514

[573 rows x 2 columns]>

In [ ]:
import math

def haversine_distance(coord1, coord2):
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    radius = 260  # Радиус Земли в километрах

    # Переводим градусы в радианы
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # Вычисляем разницу между координатами
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Вычисляем формулу Хаверсина
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance


In [ ]:
import numpy as np

def distance(lat1, lon1, lat2, lon2):
    # Переводим градусы в радианы
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Вычисляем разницу между координатами
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Вычисляем расстояние с помощью формулы гаверсинусов
    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = 6371 * c  # Радиус Земли в километрах

    return distance

# Пример использования функции
latitudes = [55.751244, 37.618423, 48.856613]
longitudes = [37.618423, 55.751244, 2.352222]

# Создаем матрицу расстояний
dist_matrix = np.zeros((len(latitudes), len(latitudes)))
for i in range(len(latitudes)):
    for j in range(len(latitudes)):
        dist_matrix[i, j] = distance(latitudes[i], longitudes[i], latitudes[j], longitudes[j])

print(dist_matrix)


[[   0.         2428.96333211 2486.28933179]
 [2428.96333211    0.         4404.41673833]
 [2486.28933179 4404.41673833    0.        ]]


Муравьиный алгоритм по координатам доставки

In [ ]:
coordinates = 'coordinates.xlsx'

In [ ]:
[[61.405797	,55.263773 ], [61.374230, 55.148575], [61.401736, 55.171504] ,[61.360503, 55.189028]]

[[61.405797, 55.263773],
 [61.37423, 55.148575],
 [61.401736, 55.171504],
 [61.360503, 55.189028]]

In [ ]:
# Загрузка данных из файла XLSX в DataFrame
data = pd.read_excel('/content/coordinates.xlsx')

# Отображение данных
print(data)
print(data.head())

       ДОЛГОТА     ШИРОТА
0    60.881252  56.782632
1    60.881252  56.782632
2    60.881252  56.782632
3    60.881252  56.782632
4    60.881252  56.782632
..         ...        ...
568  60.601138  56.870514
569  60.601138  56.870514
570  60.601138  56.870514
571  60.601138  56.870514
572  60.601138  56.870514

[573 rows x 2 columns]
     ДОЛГОТА     ШИРОТА
0  60.881252  56.782632
1  60.881252  56.782632
2  60.881252  56.782632
3  60.881252  56.782632
4  60.881252  56.782632


In [ ]:
if np.isnan(probs).any():
    # Замена NaN значений на среднее значение по столбцу
    mean_values = np.nanmean(probs, axis=0)
    probs[np.isnan(probs)] = mean_values[np.isnan(probs)]


In [ ]:
s = pd.Series(['abc'])
df = pd.DataFrame({'A': [1, 2, 3]})

result = pd.concat([s, df], axis=1)

In [ ]:
# Создание DataFrame
dataframe = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})

s = pd.DataFrame(series, columns=['A'])
df = pd.DataFrame({'A': [1, 2, 3]})
result = pd.concat([s, df], axis=1)


In [ ]:
import numpy as np
import pandas as pd

# Assuming dist_matrix and coordinates are defined appropriately

# Check for NaN values in the probs array
if np.isnan(probs).any():
    # Replace NaN values with 0.0
    probs[np.isnan(probs)] = 0.0

# Concatenate the two DataFrames along the columns (axis=1)
result = pd.concat([df1, df2], axis=1)

# Concatenate the two Series
result_series = pd.concat([series1, series2])

# Create DataFrame df
df = pd.DataFrame({'A': [1, 2, 3]})

# Concatenate Series and DataFrame horizontally
result_df_series = pd.concat([series1, df], axis=1)

# Create DataFrame coordinates
coordinates = pd.DataFrame({'X': [4, 5, 6], 'Y': [7, 8, 9]})

# Concatenate DataFrame df and coordinates horizontally
dist_matrix_with_coordinates = pd.concat([df, coordinates], axis=1)

# Fill missing values in dist_matrix_with_coordinates with 0
dist_matrix_with_coordinates = dist_matrix_with_coordinates.fillna(0)

# Compute the distance matrix using cdist
dist_matrix = cdist(coordinates, coordinates, metric='euclidean')

# Set the number of ants, iterations, and algorithm parameters
num_ants = 10
num_iterations = 100
alpha = 1.0
beta = 2.0
rho = 0.5

# Initialize pheromones on edges
pheromone = np.ones(dist_matrix.shape) / len(dist_matrix)

# Iterate through the iterations
for iteration in range(num_iterations):
    # Create an empty list to store ant paths
    ant_paths = []
    # Iterate through the ants
for ant in range(num_ants):
    # Randomly select the starting city for the ant
    start_city = np.random.randint(len(dist_matrix))
    visited_cities = [start_city]

    # Continue until all cities are visited
    while len(visited_cities) < len(dist_matrix):
        # Calculate transition probabilities to the next city
        probs = []
        current_city = visited_cities[-1]

        for next_city in range(len(dist_matrix)):
            if next_city not in visited_cities:
                pheromone_level = pheromone[current_city][next_city]
                distance = dist_matrix[current_city][next_city]
                prob = (pheromone_level ** alpha) * ((1.0 / distance) ** beta)
                probs.append(prob)
            else:
                probs.append(0.0)




            # Check for NaN values in the probability array
        if np.isnan(probs).any():
            probs[np.isnan(probs)] = 0.0  # Replace NaN with 0.0
            probs = probs[~np.isnan(probs)]  # Remove NaN values

        # Add a small constant value to each element of the probabilities list
        probs = [p + 1e-8 for p in probs]

        # Normalize the probabilities to sum up to 1
        sum_probs = sum(probs)
        probs = [p / sum_probs for p in probs]

        # Select the next city based on the probabilities
        next_city = np.random.choice(range(len(dist_matrix)), p=probs)
        visited_cities.append(next_city)

            # Select the next city based on the probabilities
        next_city = np.random.choice(range(len(dist_matrix)), p=probs)
        visited_cities.append(next_city)

        # Add the ant's path to the list of paths
        ant_paths.append(visited_cities)

    # Update pheromones on edges
    delta_pheromone = np.zeros(dist_matrix.shape)

    for path in ant_paths:
        for i in range(len(path) - 1):
            current_city = path[i]
            next_city = path[i + 1]
            delta_pheromone[current_city][next_city] += 1.0 / dist_matrix[current_city][next_city]

    pheromone = (1.0 - rho) * pheromone + delta_pheromone

# Find the best path
best_path = ant_paths[np.argmin([sum([dist_matrix[path[i]][path[i + 1]] for i in range(len(path) - 1)]) for path in ant_paths])]

print("Best path:", best_path)

Best path: [1, 0, 0]


<ipython-input-46-d65b60e2c928>:103: RuntimeWarning: divide by zero encountered in double_scalars
  delta_pheromone[current_city][next_city] += 1.0 / dist_matrix[current_city][next_city]
